In [1]:
from game import Game2048
from game_env import Game2048Env

# Search content

In [ ]:
# Definir modelo de NN por reforço
    # Métodos
        # Q-Learning
        # Monte Carlo
        # Gradiente de políticas
        # Aprendizado por diferença temporal
        # Trust Region Policy Optimization (TRPO)
# Definir parâmetros
    # Grid do jogo
# Definir penalidades
    # Tentar mover para uma direção sem movimentos válidos
    # Valores iguais/próximos muito dispersos? (viés?)
        # Aplicar penalidade apenas quando saiu do range (até 2 de distância?)
        # Verificar valores disponíveis no grid e contabilizar como valor próximo se estiver a até (3?) valores de distância
        # Não penalizar se valor for 2 ou 4.
# Definir recompensas
    # Juntar tiles
    # Manter valores da sequência em fileira ou próximos? (viés?)
    # 
# Definir estratégia de treino
    # Utilizar jogo já iniciado com mais tiles no grid? (aumentar aprendizado inicial sobre a regra do jogo)
        # Treinar o mesmo jogo com rotações diferentes

# Material:
# https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc
# https://www.analyticsvidhya.com/blog/2019/04/introduction-deep-q-learning-python
# https://www.analyticsvidhya.com/blog/2019/01/monte-carlo-tree-search-introduction-algorithm-deepmind-alphago
# https://medium.com/@samina.amin/deep-q-learning-dqn-71c109586bae
# https://www.geeksforgeeks.org/implementing-deep-q-learning-using-tensorflow/

# Model

In [2]:
# pip install tensorflow==2.9.0 keras-rl2
# pip uninstall numpy -y
# pip install numpy==1.24.3

import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

2025-03-23 12:04:07.122262: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-23 12:04:07.122299: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:

nActions = 4
tableSize = 4
hiddenFeatures = 16
# temporalWindow = 5
temporalWindow = 1
env = Game2048Env()

model = Sequential()
model.add(Flatten(input_shape=(temporalWindow, tableSize, tableSize)))

model.add(Dense(hiddenFeatures))
model.add(Activation(activation="relu"))
# model.add(Activation(activation="leaky_relu")) # Tune alpha
# model.add(Activation(activation="elu")) # Tune alpha

model.add(Dense(nActions))
model.add(Activation(activation="linear"))

policy = EpsGreedyQPolicy()
replayMemory = SequentialMemory(limit=100, window_length=temporalWindow)
dqn = DQNAgent(
    model=model,
    nb_actions=nActions,
    memory=replayMemory,
    nb_steps_warmup=1000,
    target_model_update=0.01,
    policy=policy
)
dqn.compile(optimizer=Adam(lr=0.001), metrics=["mae"])
dqn.fit(env=env, nb_steps=20000, visualize=False, verbose=2)


/usr/local/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2025-03-23 12:04:10.934372: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-03-23 12:04:10.934412: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-03-23 12:04:10.934429: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a7476f3fb4ff): /proc/driver/nvidia/version does not exist
2025-03-23 12:04:10.934641: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA


Training for 2000 steps ...


/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  294/2000: episode: 1, duration: 0.291s, episode steps: 294, steps per second: 1011, episode reward: -11318.000, mean reward: -38.497 [-50.000, 36.000], mean action: 2.194 [0.000, 3.000],  loss: --, mae: --, mean_q: --
  772/2000: episode: 2, duration: 0.292s, episode steps: 478, steps per second: 1639, episode reward: -16120.000, mean reward: -33.724 [-50.000, 128.000], mean action: 1.994 [0.000, 3.000],  loss: --, mae: --, mean_q: --


/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 1010/2000: episode: 3, duration: 0.391s, episode steps: 238, steps per second: 609, episode reward: -6612.000, mean reward: -27.782 [-50.000, 72.000], mean action: 2.113 [0.000, 3.000],  loss: 819.047926, mae: 9.188615, mean_q: 1.666015
 1436/2000: episode: 4, duration: 1.658s, episode steps: 426, steps per second: 257, episode reward: -12518.000, mean reward: -29.385 [-50.000, 128.000], mean action: 1.176 [0.000, 3.000],  loss: 666.428528, mae: 15.070562, mean_q: -6.655188
 1837/2000: episode: 5, duration: 1.555s, episode steps: 401, steps per second: 258, episode reward: -9182.000, mean reward: -22.898 [-50.000, 276.000], mean action: 1.793 [0.000, 3.000],  loss: 511.274261, mae: 43.865234, mean_q: -43.538116
done, took 4.814 seconds


# Simulate Game

In [4]:
# Play the game with predicts

def log2_transform(grid):
    transformed = np.where(grid > 0, np.log2(grid), 0)
    return transformed

actionMapping = {
    0: "up",
    1: "down",
    2: "left",
    3: "right"
}

games = [Game2048() for _ in range(10)]
metrics = {
    "gameId": [],
    "score": [],
    "higherTile": [],
    "invalidMove (%)": [],
    "steps": [],
    "upMoves": [],
    "downMoves": [],
    "leftMoves": [],
    "rightMoves": [],
    "upSeqAvg": [],
    "downSeqAvg": [],
    "leftSeqAvg": [],
    "rightSeqAvg": [],
}
for i, game in enumerate(games):
    # Metrics
    steps = 0
    invalidMove = 0
    upMoves = 0
    downMoves = 0
    leftMoves = 0
    rightMoves = 0
    upSeqAvg = 0
    downSeqAvg = 0
    leftSeqAvg = 0
    rightSeqAvg = 0

    sequences = {
        "up": [],
        "down": [],
        "left": [],
        "right": []
    }
    lastMove = None

    # print(f"Game {i + 1} initial state:")
    # game.render()
    while (not game.game_over):
        observation = np.expand_dims(game.grid, axis=0)[0]
        observation = log2_transform(observation)
        predictedAction = dqn.forward(observation)
        
        # print(f"\nPredicted Action: move {actionMapping[predictedAction]}")
        if not game.move(actionMapping[predictedAction]):
            invalidMove += 1
        
        # print(f"Game {i + 1} currrent state:")
        # game.render()

        # input("\nPress any key to continue...")

        steps += 1
        if actionMapping[predictedAction] == "up":
            upMoves += 1
            if lastMove == "up":
                sequences["up"][-1] += 1
            else:
                sequences["up"].append(1)
        elif actionMapping[predictedAction] == "down":
            downMoves += 1
            if lastMove == "down":
                sequences["down"][-1] += 1
            else:
                sequences["down"].append(1)
        elif actionMapping[predictedAction] == "left":
            leftMoves += 1
            if lastMove == "left":
                sequences["left"][-1] += 1
            else:
                sequences["left"].append(1)
        elif actionMapping[predictedAction] == "right":
            rightMoves += 1
            if lastMove == "right":
                sequences["right"][-1] += 1
            else:
                sequences["right"].append(1)
        
        lastMove = actionMapping[predictedAction]

    print(f"Game {i + 1} is over.")

    metrics["gameId"].append(i)
    metrics["score"].append(game.score)
    metrics["higherTile"].append(np.max(game.grid))
    metrics["invalidMove (%)"].append(invalidMove/steps)
    metrics["steps"].append(steps)
    metrics["upMoves"].append(upMoves)
    metrics["downMoves"].append(downMoves)
    metrics["leftMoves"].append(leftMoves)
    metrics["rightMoves"].append(rightMoves)
    metrics["upSeqAvg"].append(np.mean(np.array(sequences["up"])))
    metrics["downSeqAvg"].append(np.mean(np.array(sequences["down"])))
    metrics["leftSeqAvg"].append(np.mean(np.array(sequences["left"])))
    metrics["rightSeqAvg"].append(np.mean(np.array(sequences["right"])))


metrics = pd.DataFrame(metrics).set_index("gameId")
display(metrics)
display(metrics.describe())


/tmp/ipykernel_13702/845857735.py:4: RuntimeWarning: divide by zero encountered in log2
  transformed = np.where(grid > 0, np.log2(grid), 0)


Game 1 is over.
Game 2 is over.
Game 3 is over.
Game 4 is over.
Game 5 is over.
Game 6 is over.
Game 7 is over.
Game 8 is over.
Game 9 is over.
Game 10 is over.


,score,higherTile,invalidMove (%),steps,upMoves,downMoves,leftMoves,rightMoves,upSeqAvg,downSeqAvg,leftSeqAvg,rightSeqAvg
gameId,,,,,,,,,,,,
0,948,64,0.731308,428,9,102,288,29,1.285714,4.250000,7.783784,1.000000
1,1232,128,0.575758,297,6,63,191,37,1.000000,2.739130,3.979167,1.057143
2,736,64,0.678689,305,4,26,253,22,1.333333,1.444444,7.666667,1.047619
3,964,128,0.509434,212,9,23,102,78,1.000000,1.210526,3.000000,2.363636
4,720,64,0.741047,363,9,112,217,25,1.000000,5.333333,6.575758,1.041667
5,1372,128,0.688841,466,11,208,208,39,1.000000,5.777778,4.333333,1.000000
6,544,64,0.625616,203,3,17,156,27,1.000000,1.214286,5.032258,1.000000
7,1232,128,0.665761,368,9,59,246,54,1.000000,2.458333,4.920000,1.421053
8,1928,256,0.754304,639,23,105,399,112,1.045455,2.692308,7.823529,3.200000


,score,higherTile,invalidMove (%),steps,upMoves,downMoves,leftMoves,rightMoves,upSeqAvg,downSeqAvg,leftSeqAvg,rightSeqAvg
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,1027.200000,108.800000,0.630934,340.800000,8.700000,72.800000,214.300000,45.000000,1.066450,2.820347,5.379191,1.416958
std,424.665306,60.715731,0.128050,148.295351,5.716448,60.874735,91.825983,29.044028,0.129380,1.742945,1.965435,0.756233
min,544.000000,64.000000,0.338583,127.000000,3.000000,13.000000,83.000000,22.000000,1.000000,1.083333,2.677419,1.000000
25%,724.000000,64.000000,0.588222,233.250000,4.500000,23.750000,164.750000,27.000000,1.000000,1.271825,4.067708,1.009615
50%,956.000000,96.000000,0.672225,334.000000,9.000000,61.000000,212.500000,33.000000,1.000000,2.575321,4.976129,1.044643
75%,1232.000000,128.000000,0.720692,413.000000,9.000000,104.250000,251.250000,50.250000,1.034091,3.872283,7.393939,1.330075
max,1928.000000,256.000000,0.754304,639.000000,23.000000,208.000000,399.000000,112.000000,1.333333,5.777778,7.823529,3.200000


# Manual Game

In [ ]:
moves = ["up", "left", "right", "down"]
directions = ["w", "a", "d", "s"]

games = [Game2048() for _ in range(1)]
for i, game in enumerate(games):
    print(f"Game {i + 1} initial state:")
    game.render()
    while (not game.game_over):
        try: 
            currentMove = moves[directions.index(input("Next move (w, a, s , d): "))]
        except ValueError:
            print("Invalid move, try again.")
            continue

        game.move(currentMove)
        print(f"Game {i + 1} after move '{currentMove}':")
        game.render()

    print(f"Game {i + 1} is over.")


